In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import date
import import_ipynb
import covid_dict as cd

importing Jupyter notebook from covid_dict.ipynb


1) [Remover colunas dos atributos desconsiderados](#remove_column)  
2) [Arrumando os dados](#data_fix)  
3) [Gerando a matrix binária](#binary_matrix)  
3.1) [Matriz: Óbitos](#binary_matrix_obito)   
3.2) [Matriz: Recuperados](#binary_matrix_rec)  
3.3) [Matriz: Assintomáticos](#binary_matrix_assin)  
3.4) [Matriz: Sintomáticos](#binary_matrix_sin)  

[Panda](Panda_dataset.ipynb)

[Dicionário de dados](covid_dict.ipynb)

In [ ]:
# csv_url = "https://ti.saude.rs.gov.br/covid19/download"
#today = date.today()
# dataset = f'dataset-{today}.csv'

# req = requests.get(csv_url)
# url_content = req.content
# csv_file = open(dataset, 'wb')

# csv_file.write(url_content)
# csv_file.close()

In [2]:
df = pd.read_csv('20201030.csv', sep=';')  #nrows=1000

Atributos desconsiderados:
* COD_IBGE
* MUNICIPIO
* COD_REGIAO_COVID
* DATA_SINTOMAS
* DATA_EVOLUCAO
* DATA_INCLUSAO_OBITO
* DATA_EVOLUCAO_ESTIMADA
* ETNIA_INDIGENA
* FONTE_INFORMACAO
* PAIS_NASCIMENTO
* BAIRRO

#### 1) Remover colunas dos atributos desconsiderados <a name="remove_column"></a>

In [10]:
for e in cd.attr_removed2:
    del df[e]

#### 2) Filtrando registros -- EVOLUCAO == OBITO

In [11]:
df = df[df.eq('OBITO').any(axis=1)]

#### 3) Arrumando os dados - FAIXAETARIA, DATA_CONFIRMACAO, CONDICOES <a name="data_fix"></a>

Faixa Etária
* < 1: Bebê
* (01,09): Criança
* (10, 19): Jovem
* (20, 39): Adulto 1
* (40, 49): Adulto 2
* (50, 59): Adulto 3
* (60, 69): Idoso 1
* (70, 79): Idoso 2
* (80 +): Idoso 3
       

In [15]:
for index, row in df.iterrows():
    if row['FAIXAETARIA'] == '<1':
        row['FAIXAETARIA'] = 'Bebê'
    elif row['FAIXAETARIA'] == '01 a 04' or row['FAIXAETARIA'] == '05 a 09':
        row['FAIXAETARIA'] = 'Criança'
    elif row['FAIXAETARIA'] == '10 a 14' or row['FAIXAETARIA'] == '15 a 19':
        row['FAIXAETARIA'] = 'Jovem'
    elif row['FAIXAETARIA'] == '20 a 29' or row['FAIXAETARIA'] == '30 a 39':
        row['FAIXAETARIA'] = 'Adulto_1'
    elif row['FAIXAETARIA'] == '40 a 49':
        row['FAIXAETRIA'] = 'Adulto_2'
    elif row['FAIXAETARIA'] == '50 a 59':
        row['FAIXAETARIA'] = 'Adulto_3'
    elif row['FAIXAETARIA'] == '60 a 69':
        row['FAIXAETARIA'] = 'Idoso_1'
    elif row['FAIXAETARIA'] == '70 a 79':
        row['FAIXAETARIA'] = 'Idoso_2'
    else:
        row['FAIXAETARIA'] = 'Idoso_3'

Data de Confirmação

In [16]:
for index, row in df.iterrows():
    mes = row['DATA_CONFIRMACAO'].split('/')
    if mes[1] == '02':
        row['DATA_CONFIRMACAO'] = 'Fevereiro'
    elif mes[1] == '03':
        row['DATA_CONFIRMACAO'] = 'Março'
    elif mes[1] == '04':
        row['DATA_CONFIRMACAO'] = 'Abril'
    elif mes[1] == '05':
        row['DATA_CONFIRMACAO'] = 'Maio'    
    elif mes[1] == '06':
        row['DATA_CONFIRMACAO'] = 'Junho'
    elif mes[1] == '07':
        row['DATA_CONFIRMACAO'] = 'Julho'
    elif mes[1] == '08':
        row['DATA_CONFIRMACAO'] = 'Agosto'
    elif mes[1] == '09':
        row['DATA_CONFIRMACAO'] = 'Setembro'
    else:
        row['DATA_CONFIRMACAO'] = 'Outubro'

Condições (Comorbidades)

In [17]:
for e in cd.comorbidades:
    df[e] = str(np.nan)
    
for index, row in df.iterrows():
    if 'asma' in str(row['CONDICOES']).lower():
        row['asma'] = 'SIM'
    else:
        row['asma'] = 'NAO'
    if 'respirató' in str(row['CONDICOES']).lower():
        row['d_respiratoria'] = 'SIM'
    else:
        row['d_respiratoria'] = 'NAO'
    if 'outro' in str(row['CONDICOES']).lower():
        row['outras_comorbidades'] = 'SIM'
    else:
        row['outras_comorbidades'] = 'NAO'
    if 'outro' in str(row['CONDICOES']).lower():
        row['outras_comorbidades'] = 'SIM'
    else:
        row['outras_comorbidades'] = 'NAO'
    if 'cromossômicas' in str(row['CONDICOES']).lower():
        row['d_cromossomica'] = 'SIM'
    else:
        row['d_cromossomica'] = 'NAO'
    if 'neuro' in str(row['CONDICOES']).lower():
        row['d_neurologica'] = 'SIM'
    else:
        row['d_neurologica'] = 'NAO'
    if 'diabe' in str(row['CONDICOES']).lower():
        row['diabetes'] = 'SIM'
    else:
        row['diabetes'] = 'NAO'
    if 'imuno' in str(row['CONDICOES']).lower():
        row['imunodeficiencia'] = 'SIM'
    else:
        row['imunodeficiencia'] = 'NAO'
    if 'rena' in str(row['CONDICOES']).lower():
        row['d_renal'] = 'SIM'
    else:
        row['d_renal'] = 'NAO'
    if 'hepát' in str(row['CONDICOES']).lower():
        row['d_hepatica'] = 'SIM'
    else:
        row['d_hepatica'] = 'NAO'
    if 'hemato' in str(row['CONDICOES']).lower():
        row['d_hematologica'] = 'SIM'
    else:
        row['d_hematologica'] = 'NAO'
    if 'card' in str(row['CONDICOES']).lower():
        row['d_cardiaca'] = 'SIM'
    else:
        row['d_cardiaca'] = 'NAO'
    if 'obes' in str(row['CONDICOES']).lower():
        row['obesidade'] = 'SIM'
    else:
        row['obesidade'] = 'NAO'
    if 'down' in str(row['CONDICOES']).lower():
        row['sindrome_down'] = 'SIM'
    else:
        row['sindrome_down'] = 'NAO'
del df['CONDICOES']

### Gerando a matrix binária <a name="binary_matrix"></a>

In [18]:
for e in cd.regiao_covid:
    df[e] = df.apply(lambda row: 1 if row.REGIAO_COVID == e else 0, axis=1)
    
for e in cd.faixa_etaria2:
    df[e] = df.apply(lambda row: 1 if row.FAIXAETARIA == e else 0, axis=1)
    
for e in cd.testagem: 
    df[e] = df.apply(lambda row: 1 if row.CRITERIO == e else 0, axis=1)

for e in cd.meses:
    df[e] = df.apply(lambda row: 1 if row.DATA_CONFIRMACAO == e else 0, axis=1)
        
for e in cd.etnia:
    df[e] = df.apply(lambda row: 1 if row.RACA_COR == e else 0, axis=1)

In [19]:
df['SexF'] = df.apply(lambda row: 1 if row.SEXO == 'Feminino' else 0, axis=1)
df['SexM'] = df.apply(lambda row: 1 if row.SEXO == 'Masculino' else 0, axis=1)
df['Hospitalizado'] = df.apply(lambda row: 1 if row.HOSPITALIZADO == 'SIM' else 0, axis=1)
df['Hosp_SRAG'] = df.apply(lambda row: 1 if row.HOSPITALIZACAO_SRAG == 'SIM' else 0, axis=1)
df['Prof_Saude'] = df.apply(lambda row: 1 if row.PROFISSIONAL_SAUDE == 'SIM' else 0, axis=1)
df['Febre'] = df.apply(lambda row: 1 if row.FEBRE == 'SIM' else 0, axis=1)
df['Tosse'] = df.apply(lambda row: 1 if row.TOSSE == 'SIM' else 0, axis=1)
df['Garganta'] = df.apply(lambda row: 1 if row.GARGANTA == 'SIM' else 0, axis=1)
df['Dispneia'] = df.apply(lambda row: 1 if row.DISPNEIA == 'SIM' else 0, axis=1)
df['Outros_Sint'] = df.apply(lambda row: 1 if row.OUTROS == 'SIM' else 0, axis=1)
df['Gestante'] = df.apply(lambda row: 1 if row.GESTANTE == 'SIM' else 0, axis=1)
df['Presidiario'] = df.apply(lambda row: 1 if row.PES_PRIV_LIBERDADE == 'SIM' else 0, axis=1)
df['Asma'] = df.apply(lambda row: 1 if row.asma == 'SIM' else 0, axis=1)
df['DRespiratoria'] = df.apply(lambda row: 1 if row.d_respiratoria == 'SIM' else 0, axis=1)
df['DNeurologica'] = df.apply(lambda row: 1 if row.d_neurologica == 'SIM' else 0, axis=1)
df['Diabetes'] = df.apply(lambda row: 1 if row.diabetes == 'SIM' else 0, axis=1)
df['Imunodef'] = df.apply(lambda row: 1 if row.imunodeficiencia == 'SIM' else 0, axis=1)
df['DRenal'] = df.apply(lambda row: 1 if row.d_renal == 'SIM' else 0, axis=1)
df['DHepatica'] = df.apply(lambda row: 1 if row.d_hepatica == 'SIM' else 0, axis=1)
df['DCardiaca'] = df.apply(lambda row: 1 if row.d_cardiaca == 'SIM' else 0, axis=1)
df['Obesidade'] = df.apply(lambda row: 1 if row.obesidade == 'SIM' else 0, axis=1)
df['Outras_Comorbidades'] = df.apply(lambda row: 1 if row.outras_comorbidades == 'SIM' else 0, axis=1)
df['DCromossomica'] = df.apply(lambda row: 1 if row.d_cromossomica == 'SIM' else 0, axis=1)
df['DHematologica'] = df.apply(lambda row: 1 if row.d_hematologica == 'SIM' else 0, axis=1)
df['Sindrome_down'] = df.apply(lambda row: 1 if row.sindrome_down == 'SIM' else 0, axis=1)

In [ ]:
for e in cd.old_columns2:
    del df[e]

In [25]:
df.to_csv("20201030_binary_matrix(covid_region).csv", header=None, index=False)

In [26]:
nome_atributos = df.columns.values.tolist()
df_dict = {i : nome_atributos[i] for i in range(0, len(nome_atributos))}

In [27]:
df.to_csv("20201030_binary_matrix(header)(covid_region).csv", index=False)

In [28]:
df.columns

Index(['BAGE - R22', 'PASSO FUNDO - R17 R18 R19', 'SANTA MARIA - R01 R02',
       'IJUI - R13', 'SANTA ROSA - R14', 'URUGUAIANA - R03',
       'PALMEIRA DAS MISSOES - R15 R20', 'CAXIAS DO SUL - R23 R24 R25 R26',
       'PORTO ALEGRE - R10', 'PELOTAS - R21', 'LAJEADO - R29 R30',
       'GUAIBA - R09', 'NOVO HAMBURGO - R07', 'ERECHIM - R16',
       'CAPAO DA CANOA - R04 R05', 'CACHOEIRA DO SUL - R27', 'CANOAS - R08',
       'CRUZ ALTA - R12', 'SANTO ANGELO - R11', 'TAQUARA - R06',
       'SANTA CRUZ DO SUL - R28', 'Bebê', 'Criança', 'Jovem', 'Adulto_1',
       'Adulto_2', 'Adulto_3', 'Idoso_1', 'Idoso_2', 'Idoso_3', 'RT-PCR',
       'TESTE RÁPIDO', 'Outros Testes', 'Clínico-Imagem', 'Clínico',
       'Clínico Epidemiológico', 'Março', 'Abril', 'Maio', 'Junho', 'Julho',
       'Agosto', 'Setembro', 'Outubro', 'BRANCA', 'NAO INFORMADO', 'INDIGENA',
       'AMARELA', 'PARDA', 'PRETA', 'SexF', 'SexM', 'Hospitalizado',
       'Hosp_SRAG', 'Prof_Saude', 'Febre', 'Tosse', 'Garganta', 'Dispneia'

In [29]:
df_dict

{0: 'BAGE - R22',
 1: 'PASSO FUNDO - R17 R18 R19',
 2: 'SANTA MARIA - R01 R02',
 3: 'IJUI - R13',
 4: 'SANTA ROSA - R14',
 5: 'URUGUAIANA - R03',
 6: 'PALMEIRA DAS MISSOES - R15 R20',
 7: 'CAXIAS DO SUL - R23 R24 R25 R26',
 8: 'PORTO ALEGRE - R10',
 9: 'PELOTAS - R21',
 10: 'LAJEADO - R29 R30',
 11: 'GUAIBA - R09',
 12: 'NOVO HAMBURGO - R07',
 13: 'ERECHIM - R16',
 14: 'CAPAO DA CANOA - R04 R05',
 15: 'CACHOEIRA DO SUL - R27',
 16: 'CANOAS - R08',
 17: 'CRUZ ALTA - R12',
 18: 'SANTO ANGELO - R11',
 19: 'TAQUARA - R06',
 20: 'SANTA CRUZ DO SUL - R28',
 21: 'Bebê',
 22: 'Criança',
 23: 'Jovem',
 24: 'Adulto_1',
 25: 'Adulto_2',
 26: 'Adulto_3',
 27: 'Idoso_1',
 28: 'Idoso_2',
 29: 'Idoso_3',
 30: 'RT-PCR',
 31: 'TESTE RÁPIDO',
 32: 'Outros Testes',
 33: 'Clínico-Imagem',
 34: 'Clínico',
 35: 'Clínico Epidemiológico',
 36: 'Março',
 37: 'Abril',
 38: 'Maio',
 39: 'Junho',
 40: 'Julho',
 41: 'Agosto',
 42: 'Setembro',
 43: 'Outubro',
 44: 'BRANCA',
 45: 'NAO INFORMADO',
 46: 'INDIGENA',
 4